In [9]:
# -*- coding: utf-8 -*-

In [10]:
import os
import pandas as pd 
from glob import glob

import sys
import time, re, pickle, itertools
import urllib3, json
sys.path.append('~/Documents/GitHub/Private_Project/personal_project/')
import re

import nltk
from __future__ import print_function, unicode_literals
from gensim.models.word2vec import Word2Vec
from sklearn.preprocessing import normalize as sknorm
import numpy as np
from operator import itemgetter
import html

from ckonlpy.tag import Twitter as ctwitter
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
import numpy as np
import matplotlib.pyplot as plt
from konlpy.tag import Twitter, Mecab,Kkma
from konlpy.utils import pprint

import chat_bot as cb
import Database_Handler as dh

ct = ctwitter()
ot = Twitter()
mecab = Mecab()
xxxx = ct._dictionary._pos2words
print (list(map(lambda x: len(xxxx[x]), xxxx)))

score_weights = {
    'num_nouns': -0.1,
    'num_words': -0.2,
    'no_noun': -1
}

def my_score(candidate):
    num_nouns = len([w for w,t in candidate if t == 'Noun'])
    num_words = len(candidate)
    no_noun = 1 if num_nouns == 0 else 0
    
    score = (num_nouns * score_weights['num_nouns'] 
             + num_words * score_weights['num_words']
             + no_noun * score_weights['no_noun'])
    return score

ct.set_selector(score_weights, my_score)

[512, 1384828, 5532]
[('이', 'Noun'), ('것', 'Noun'), ('은', 'Josa'), ('테', 'Noun'), ('스트', 'Noun')]
[('이것', 'Noun'), ('은', 'Josa'), ('테', 'Noun'), ('스트', 'Noun')]
[('이것', 'Noun'), ('은', 'Josa'), ('테스트', 'Noun')]
best: [('이것', 'Noun'), ('은', 'Josa'), ('테스트', 'Noun')]


# 자연어 처리

## TextRank 

In [11]:
stopwords = open('./data/stopwordsList.txt','r',encoding='utf-8').readlines() + open('./data/newspress.txt','r',encoding = 'utf-8').readlines()
stopwords = list(map(lambda x: x.strip(), stopwords))

참고
[인용사이트](http://excelsior-cjh.tistory.com/entry/TextRank%EB%A5%BC-%EC%9D%B4%EC%9A%A9%ED%95%9C-%EB%AC%B8%EC%84%9C%EC%9A%94%EC%95%BD)

In [12]:

class SentenceTokenizer(object):
    def __init__(self):
        self.twitter = ot
        self.ctwitter = ct
        self.mecab = mecab
        
        #self.stopwords = []
        self.stopwords = stopwords
    def text2sentences(self, text):
        sentences = re.sub('\. ','.\n', text).split('\n')
        for idx in range(0, len(sentences)):
            if len(sentences[idx]) <= 10:
                sentences[idx-1] += (' ' + sentences[idx])
                sentences[idx] = ''
        return sentences
    def get_nouns(self, sentences):
        nouns = []
        for sentence in sentences:
            if sentence is not '':
                nouns.append(' '.join([noun for noun in self.ctwitter.phrases(str(sentence))
                                                    if noun not in self.stopwords and len(noun) > 1]))
                nouns.append(' '.join([noun for noun in self.ctwitter.nouns(str(sentence))
                                                  if noun not in self.stopwords and len(noun) > 1]))
                nouns.append(' '.join([noun for noun in self.ctwitter.morphs(str(sentence))
                                                   if noun not in self.stopwords and len(noun) > 1]))
                #nouns.append(' '.join([noun for noun in self.mecab.nouns(str(sentence))
                #                                  if noun not in self.stopwords and len(noun) > 1]))
                #nouns.append(' '.join([noun for noun in self.mecab.phrases(str(sentence))
                #                                  if noun not in self.stopwords and len(noun) > 1]))
                #nouns.append(' '.join([noun for noun in self.mecab.morphs(str(sentence))
                #                                   if noun not in self.stopwords and len(noun) > 1]))
        nouns = list(set(nouns))
        return nouns
    
class GraphMatrix(object):
    def __init__(self):
        self.tfidf = TfidfVectorizer(ngram_range=(1,5),sublinear_tf=True, lowercase=False,min_df=0.000001)
        self.cnt_vec = CountVectorizer(ngram_range=(1,5), lowercase=False,min_df=0.000001)
        self.graph_sentence = []
    def build_sent_graph(self, sentence):
        tfidf_mat = self.tfidf.fit_transform(sentence).toarray()
        #tfidf_mat = self.tfidf.fit_transform(sentence).toarray()
        self.graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
        return self.graph_sentence
    def build_words_graph(self, sentence):
        cnt_vec_mat = sknorm(self.cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
        vocab = self.cnt_vec.vocabulary_
        return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}

class Rank(object):
    def get_ranks(self, graph, d=0.85): # d = damping factor
        A = graph
        matrix_size = A.shape[0]
        for id in range(matrix_size):
            A[id, id] = 0 # diagonal 부분을 0으로
            link_sum = np.sum(A[:,id]) # A[:, id] = A[:][id]
            if link_sum != 0:
                A[:, id] /= link_sum
            A[:, id] *= -d
            A[id, id] = 1
        B = (1-d) * np.ones((matrix_size, 1))
        ranks = np.linalg.solve(A, B) # 연립방정식 Ax = b
        return {idx: r[0] for idx, r in enumerate(ranks)}

class TextRankX(object):
    def __init__(self, text):
        self.sent_tokenize = SentenceTokenizer()
        self.sentences = self.sent_tokenize.text2sentences(text)
        self.nouns = self.sent_tokenize.get_nouns(self.sentences)
        self.graph_matrix = GraphMatrix()
        self.sent_graph = self.graph_matrix.build_sent_graph(self.nouns)
        self.words_graph, self.idx2word = self.graph_matrix.build_words_graph(self.nouns)
        self.rank = Rank()
        self.sent_rank_idx = self.rank.get_ranks(self.sent_graph)
        self.sorted_sent_rank_idx = sorted(self.sent_rank_idx, key=lambda k: self.sent_rank_idx[k], reverse=True)
        self.word_rank_idx = self.rank.get_ranks(self.words_graph)
        self.sorted_word_rank_idx = sorted(self.word_rank_idx, key=lambda k: self.word_rank_idx[k], reverse=True)
    def summarize(self, sent_num=3):
        summary = []
        index=[]
        for idx in self.sorted_sent_rank_idx[:sent_num]:
            index.append(idx)
        index.sort()
        for idx in index:
            summary.append(self.sentences[idx])
        return summary
    def keywords(self, word_num=10):
        rank = Rank()
        rank_idx = rank.get_ranks(self.words_graph)
        sorted_rank_idx = sorted(rank_idx, key=lambda k: rank_idx[k], reverse=True)
        keywords = []
        index=[]
        for idx in sorted_rank_idx[:word_num]:
            index.append(idx)
        #index.sort()
        for idx in index:
            keywords.append(self.idx2word[idx])
        return keywords

# 분석

In [13]:
path = './data/Keyword_outcome/ckonlpy-naver.*'
extracted_file = glob(path)
extracted = dict()
for file in extracted_file:
    ex = pickle.load(open(file,'rb'))
    extracted.update(ex)
outlen = str(len(extracted_file))
print (len(extracted))
outname = './data/Keyword_outcome/ckonlpy-naver.pickled_'+outlen

0


In [ ]:
extract_keywords_dict = dict()
site = 'Naver'
#site = 'daum'
if site == 'daum':
    binList = []
    collection = 'newsDaum'
elif site.lower() == 'naver':
    binList = []
    collection = 'newsNaver'

mongodb = dh.ToMongoDB(*dh.AWS_MongoDB_Information())
dbname = 'hy_db'
useDb = dh.Use_Database(mongodb, dbname)
slack = cb.Slacker(cb.slacktoken())
useCollection = dh.Use_Collection(useDb, collection)
keywordList = list()
collection = useCollection.find({'site':site})
count = 0
for data in collection:
    idIs = data['_id']._ObjectId__id
    tis2 = html.unescape(data['title'])+'. '+html.unescape(data['mainText'])
    if data['keywords'] !='NaN':
        keywordList += list(map(lambda x: ''.join(x.split(' · ')[-1].split(' ')), data['keywords']))
    else:
        pass
    if not idIs.hex() in extracted:
        try:
            workrank = TextRankX(tis2)
        except:
            try:
                workrank = TextRankX(tis2)
            except:
                workrank = TextRankX(tis2)
                break
        extract_keywords_dict[idIs.hex()]= workrank.keywords()
        if count == 1500:
            break
        print (count,)
        count +=1
pickle.dump(extract_keywords_dict, open(outname,'wb'))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72


In [ ]:
pickle.dump(extract_keywords_dict, open(outname,'wb'))